# Setup

## Spark session and context

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Spark Web UI

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

for tunnel in ngrok.get_tunnels():
  ngrok.disconnect(tunnel.public_url)
ngrok.connect(4040).public_url

## Download the dataset


In [ ]:
!wget https://www.ncei.noaa.gov/pub/data/uscrn/products/hourly02/snapshots/CRNH0203-202101180550.zip

In [ ]:
!unzip CRNH0203-202101180550.zip -d data

# Introduction to RDD

In [ ]:
numbers = sc.parallelize(range(1000000))

In [ ]:
numbers.take(5)

In [ ]:
numbers.count()

In [ ]:
numbers = sc.parallelize(range(1000000)).setName("numbers").cache()
numbers.count()

In [ ]:
numbers.take(5)

In [ ]:
numbers = sc.parallelize(range(1000000)).repartition(20).setName("numbers3").cache()
numbers.count()

# First MapReduce exercises

### Sum

In [ ]:
numbers.reduce(
    lambda x, y: x + y
)

In [ ]:
numbers.sum()

### Average

In [ ]:
total, count = numbers.map(
    lambda x: (x, 1)
).reduce(
    lambda a, b: (a[0] + b[0], a[1] + b[1])
)

total / count

In [ ]:
numbers.mean()

### Maximum

In [ ]:
numbers.reduce(lambda x, y: x if x >= y else y)

In [ ]:
numbers.reduce(max)

In [ ]:
numbers.max()

# Real-life RDD example: temperatures dataset

### Load the data in an RDD

In [ ]:
header = sc.textFile("data/HEADERS.txt").map(lambda row: row.strip().split()).collect()[1]

In [ ]:
!head data/2020/CRNH0203-2020-NY_Ithaca_13_E.txt

In [ ]:
weather_rdd = sc.textFile("data/2020/*.txt").map(lambda row: row.split())

In [ ]:
weather_rdd.cache().count()

### Maximum temperature per measurment station

In [ ]:
index_station = header.index("WBANNO")
index_temperature = header.index("T_HR_AVG")

In [ ]:
temperature_maxima = weather_rdd.map(
    lambda row: (row[index_station], float(row[index_temperature]))
).reduceByKey(
    lambda a, b: max(a, b)
).collect()

### Average temperatures per place

#### Two-pass solution

In [ ]:
temperature_sums = weather_rdd.map(
    lambda row: (row[index_station], float(row[index_temperature]))
).reduceByKey(
    lambda a, b: a + b
).collect()

In [ ]:
temperature_counts = weather_rdd.map(
    lambda row: (row[index_station], 1)
).reduceByKey(
    lambda a, b: a + b
).collect()

In [ ]:
temperature_sums = dict(temperature_sums)
temperature_counts = dict(temperature_counts)
temperature_averages = {}

for station in temperature_sums.keys():
    temperature_averages[station] = temperature_sums[station] / temperature_counts[station]

#### Single-pass solution

In [ ]:
temperature_averages = weather_rdd.map(
    lambda row: (row[index_station], (float(row[index_temperature]), 1))
).reduceByKey(
    lambda a, b: (a[0] + b[0], a[1] + b[1])
).map(
    lambda x: (x[0], x[1][0] / x[1][1])
).collect()

### Temperatures histogram

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
counts = weather_rdd.map(
    lambda row: (float(row[index_temperature]), 1)
).reduceByKey(
    lambda a, b: a + b
).collect()

In [ ]:
x, y = zip(*sorted(counts)[1:])
plt.bar(x, y)

In [ ]:
counts = weather_rdd.map(
    lambda row: float(row[index_temperature])
).countByValue()

### Average, min and max temperature per place - in a single pass!

In [ ]:
temperature_stats = weather_rdd.map(
    lambda row: (
        row[index_station],
        (
            1,
            float(row[index_temperature]),
            float(row[index_temperature]),
            float(row[index_temperature]),
        )
    )
).filter(
    lambda t: t[1][1] > -9999.0
).reduceByKey(
    lambda a, b: (
        a[0] + b[0],       # running count
        a[1] + b[1],       # running sum
        min(a[2], b[2]),   # running minimum
        max(a[3], b[3]),   # running maximum
    )
).map(
    lambda t: (
        t[1][1] / t[1][0],  # average
        t[1][2],            # minimum
        t[1][3],            # maximum
    )
).collect()

# Spark SQL

## From RDD to DataFrame

In [ ]:
products_rdd = sc.parallelize([
    {'name': 'iPhone', 'price': 800.},
    {'name': 'Galaxy', 'price': 799.},
    {'name': 'Huawei', 'price': 789.},
])

In [ ]:
from pyspark.sql import Row

In [ ]:
products_rdd = sc.parallelize([
    Row(name='iPhone', price=800.),
    Row(name='Galaxy', price=799.),
    Row(name='Huawei', price=789.),
])

In [ ]:
products_df = products_rdd.toDF()

In [ ]:
products_df.toPandas()

## Temperatures dataset: Spark SQL version

In [ ]:
weather_df = weather_rdd.toDF(header)

In [ ]:
weather_df.limit(10).toPandas()

In [ ]:
weather_df.cache().count()

In [ ]:
weather_df.sample(withReplacement=True, fraction=0.0001).toPandas()

In [ ]:
weather_df = weather_df.withColumn("T_HR_AVG", weather_df["T_HR_AVG"].cast("float"))

In [ ]:
temperature_averages = weather_df.groupBy("WBANNO").mean("T_HR_AVG").toPandas()

## Query with SQL language

In [ ]:
weather_df.createOrReplaceTempView("weather")

In [ ]:
temperature_averages = spark.sql("SELECT WBANNO, avg(T_HR_AVG) FROM weather GROUP BY WBANNO").toPandas()